# Classificazione dei Testi nel Customer Care

Questo notebook illustra come implementare un sistema di classificazione automatica delle richieste dei clienti utilizzando tecniche di Natural Language Processing (NLP). Ci concentreremo su un caso d'uso tipico del customer care: la categorizzazione automatica delle email o messaggi dei clienti in diverse categorie di problemi o richieste.

## Obiettivi del Notebook

- Creare un dataset di esempio di richieste di customer care
- Preprocessare i testi per l'analisi NLP
- Implementare e confrontare diversi approcci di classificazione
- Valutare le performance dei modelli
- Applicare il modello a nuovi dati

## Caso d'uso: Classificazione delle Richieste dei Clienti

In un contesto di customer care, le aziende ricevono quotidianamente numerose richieste attraverso vari canali (email, chat, social media). Classificare automaticamente queste richieste permette di:

- Indirizzarle al reparto o all'operatore più appropriato
- Prioritizzarle in base all'urgenza o alla tipologia
- Monitorare i tipi di problemi più frequenti
- Automatizzare risposte per richieste comuni

In questo notebook, creeremo un classificatore che può categorizzare le richieste dei clienti in diverse classi come:
- Problemi tecnici
- Domande su fatturazione
- Richieste di informazioni
- Reclami
- Richieste di assistenza

## 1. Setup e Importazione delle Librerie

In [ ]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Librerie per il preprocessing del testo
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')

# Librerie per la feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Librerie per la modellazione
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Librerie per la valutazione
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Impostazioni di visualizzazione
plt.style.use('ggplot')
sns.set(style='whitegrid')

# Download delle risorse NLTK necessarie
nltk.download('punkt')
nltk.download('stopwords')

## 2. Creazione di un Dataset di Esempio

Per questo notebook, creeremo un dataset sintetico di richieste di customer care. In un contesto reale, utilizzeresti dati storici delle comunicazioni con i clienti, opportunamente anonimizzati.

In [ ]:
# Creazione di un dataset di esempio
data = {
    'testo': [
        # Problemi tecnici
        "Il mio router non si connette a internet da ieri sera. Ho già provato a riavviarlo ma non funziona.",
        "L'app continua a bloccarsi ogni volta che provo ad accedere alla sezione account. È frustrante!",
        "Non riesco ad accedere al mio account, dice che la password è errata ma sono sicuro sia corretta.",
        "La connessione internet è molto lenta negli ultimi giorni, è un problema della vostra rete?",
        "Il vostro sito web non carica correttamente le immagini dei prodotti, ho provato con diversi browser.",
        "Ho problemi con la configurazione del nuovo modem, le istruzioni non sono chiare.",
        "La TV on demand si blocca continuamente durante la riproduzione dei film.",
        "Non riesco a configurare la mail sul mio nuovo smartphone, potete aiutarmi?",
        "Il segnale della linea telefonica è disturbato, si sente un fruscio continuo durante le chiamate.",
        "L'app non si aggiorna all'ultima versione, rimane bloccata al 50% del download.",
        
        # Domande su fatturazione
        "Ho ricevuto una fattura con un importo superiore al solito, potete spiegarmi il motivo?",
        "Non capisco alcune voci nella mia ultima bolletta, in particolare il costo aggiuntivo di 15 euro.",
        "Vorrei passare alla fatturazione elettronica, come posso fare?",
        "Non ho ancora ricevuto la fattura di marzo, quando verrà emessa?",
        "Ho notato un addebito duplicato nell'estratto conto, vorrei un rimborso.",
        "Quali sono le modalità di pagamento accettate per le fatture mensili?",
        "Vorrei modificare l'indirizzo di fatturazione, ho recentemente cambiato casa.",
        "C'è un errore nel mio codice fiscale sulla fattura, come posso correggerlo?",
        "Posso rateizzare il pagamento della mia ultima bolletta? È più alta del previsto.",
        "Non ho ricevuto la conferma del pagamento dell'ultima fattura, potete verificare?",
        
        # Richieste di informazioni
        "Vorrei sapere quali sono le offerte attualmente disponibili per la fibra ottica.",
        "Quali sono gli orari di apertura del vostro negozio in centro città?",
        "Sto valutando di cambiare operatore, potete dirmi quali vantaggi offrite rispetto ai concorrenti?",
        "Il vostro servizio è disponibile anche nella mia zona? Abito a Monza.",
        "Vorrei informazioni sul nuovo smartphone appena uscito, lo avete disponibile?",
        "Quali documenti servono per attivare un nuovo contratto a nome di un'azienda?",
        "Avete un servizio di assistenza tecnica a domicilio? Quali sono i costi?",
        "Quanto tempo richiede mediamente l'attivazione di una nuova linea fissa?",
        "Vorrei sapere se è possibile mantenere il mio numero attuale cambiando piano tariffario.",
        "Quali sono le condizioni di recesso anticipato dal contratto?",
        
        # Reclami
        "Sono molto insoddisfatto del servizio clienti, ho atteso in linea per oltre 40 minuti!",
        "È la terza volta questo mese che la connessione si interrompe, è inaccettabile per un servizio premium.",
        "Ho pagato per un servizio che non funziona correttamente, voglio un rimborso immediato.",
        "Il tecnico non si è presentato all'appuntamento e nessuno mi ha avvisato, ho perso mezza giornata di lavoro.",
        "La qualità del servizio è peggiorata drasticamente negli ultimi mesi, sto valutando di cambiare operatore.",
        "Nonostante le numerose segnalazioni, il problema persiste. Questa è l'ultima possibilità prima di rivolgermi alle associazioni consumatori.",
        "Sono stato trattato in modo scortese dall'operatore al telefono, pretendo delle scuse.",
        "Mi avete addebitato costi non previsti dal contratto, è una pratica commerciale scorretta.",
        "La pubblicità del vostro servizio è ingannevole, le prestazioni reali sono molto inferiori a quelle promesse.",
        "Ho atteso tre settimane per l'installazione, ben oltre i 5 giorni lavorativi garantiti.",
        
        # Richieste di assistenza
        "Come posso configurare la casella email sul mio nuovo iPhone?",
        "Ho bisogno di aiuto per impostare il parental control sul decoder TV.",
        "Non riesco a trovare come attivare il roaming sul mio telefono, potete guidarmi?",
        "Vorrei assistenza per trasferire i contatti dal vecchio al nuovo telefono.",
        "Come posso verificare il credito residuo della mia SIM?",
        "Ho bisogno di assistenza per configurare la VPN aziendale sul mio laptop.",
        "Potete aiutarmi a ripristinare la password del mio account online?",
        "Come si imposta la segreteria telefonica sul nuovo sistema?",
        "Ho bisogno di aiuto per collegare il mio smart TV alla rete Wi-Fi di casa.",
        "Non so come attivare la funzione di risparmio dati sul mio piano mobile, potete spiegarmi?"
    ],
    'categoria': [
        # 10 problemi tecnici
        'Problema tecnico', 'Problema tecnico', 'Problema tecnico', 'Problema tecnico', 'Problema tecnico',
        'Problema tecnico', 'Problema tecnico', 'Problema tecnico', 'Problema tecnico', 'Problema tecnico',
        
        # 10 domande su fatturazione
        'Fatturazione', 'Fatturazione', 'Fatturazione', 'Fatturazione', 'Fatturazione',
        'Fatturazione', 'Fatturazione', 'Fatturazione', 'Fatturazione', 'Fatturazione',
        
        # 10 richieste di informazioni
        'Informazioni', 'Informazioni', 'Informazioni', 'Informazioni', 'Informazioni',
        'Informazioni', 'Informazioni', 'Informazioni', 'Informazioni', 'Informazioni',
        
        # 10 reclami
        'Reclamo', 'Reclamo', 'Reclamo', 'Reclamo', 'Reclamo',
        'Reclamo', 'Reclamo', 'Reclamo', 'Reclamo', 'Reclamo',
        
        # 10 richieste di assistenza
        'Assistenza', 'Assistenza', 'Assistenza', 'Assistenza', 'Assistenza',
        'Assistenza', 'Assistenza', 'Assistenza', 'Assistenza', 'Assistenza'
    ]
}

# Creazione del DataFrame
df = pd.DataFrame(data)

# Visualizzazione delle prime righe
df.head()

In [ ]:
# Esploriamo la distribuzione delle categorie
plt.figure(figsize=(10, 6))
sns.countplot(y='categoria', data=df, order=df['categoria'].value_counts().index)
plt.title('Distribuzione delle Categorie')
plt.xlabel('Numero di Richieste')
plt.ylabel('Categoria')
plt.tight_layout()
plt.show()

## 3. Preprocessing del Testo

Prima di addestrare i modelli di classificazione, dobbiamo preprocessare i testi per renderli più adatti all'analisi. Il preprocessing tipicamente include:

- Conversione in minuscolo
- Rimozione della punteggiatura
- Tokenizzazione (divisione del testo in parole)
- Rimozione delle stopwords (parole comuni come "e", "il", "la" che non aggiungono significato)
- Stemming o lemmatizzazione (riduzione delle parole alla loro forma base)

Implementiamo una funzione di preprocessing:

In [ ]:
# Inizializzazione dello stemmer per l'italiano
stemmer = SnowballStemmer('italian')
stop_words = set(stopwords.words('italian'))

def preprocess_text(text):
    # Conversione in minuscolo
    text = text.lower()
    
    # Rimozione della punteggiatura
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    
    # Tokenizzazione
    tokens = word_tokenize(text)
    
    # Rimozione delle stopwords e stemming
    processed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words and len(token) > 2]
    
    # Ricostruzione del testo
    processed_text = ' '.join(processed_tokens)
    
    return processed_text

# Applicazione del preprocessing ai testi
df['testo_preprocessato'] = df['testo'].apply(preprocess_text)

# Visualizzazione di un esempio prima e dopo il preprocessing
esempio_idx = 0
print(f"Testo originale:\n{df['testo'][esempio_idx]}\n")
print(f"Testo preprocessato:\n{df['testo_preprocessato'][esempio_idx]}")

## 4. Feature Extraction

Per addestrare modelli di machine learning su testi, dobbiamo convertire i testi in rappresentazioni numeriche. Due approcci comuni sono:

1. **Bag of Words (BoW)**: Conta semplicemente quante volte ogni parola appare in un documento.
2. **Term Frequency-Inverse Document Frequency (TF-IDF)**: Pesa le parole in base alla loro frequenza nel documento e alla loro rarità nel corpus.

Implementiamo entrambi gli approcci e confrontiamoli:

In [ ]:
# Divisione in training e test set
X = df['testo_preprocessato']
y = df['categoria']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Bag of Words
count_vectorizer = CountVectorizer()
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Visualizzazione delle dimensioni delle feature
print(f"Dimensioni BoW: {X_train_bow.shape}")
print(f"Dimensioni TF-IDF: {X_train_tfidf.shape}")

## 5. Addestramento e Valutazione dei Modelli

Ora addestreremo diversi modelli di classificazione e valuteremo le loro performance. Utilizzeremo:

1. Naive Bayes
2. Regressione Logistica
3. Support Vector Machine (SVM)
4. Random Forest

Confronteremo le performance sia con BoW che con TF-IDF.

In [ ]:
# Funzione per addestrare e valutare un modello
def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name):
    # Addestramento
    model.fit(X_train, y_train)
    
    # Predizione
    y_pred = model.predict(X_test)
    
    # Valutazione
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f"Modello: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(report)
    
    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=sorted(y.unique()), 
                yticklabels=sorted(y.unique()))
    plt.title(f'Matrice di Confusione - {model_name}')
    plt.ylabel('Valore Reale')
    plt.xlabel('Valore Predetto')
    plt.tight_layout()
    plt.show()
    
    return model, accuracy

In [ ]:
# Modelli da testare
models = {
    'Naive Bayes': MultinomialNB(),
    'Regressione Logistica': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': LinearSVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# Risultati per confronto
results_bow = []
results_tfidf = []

# Addestramento e valutazione con BoW
print("\n=== Risultati con Bag of Words ===\n")
for name, model in models.items():
    _, accuracy = train_and_evaluate(model, X_train_bow, X_test_bow, y_train, y_test, f"{name} (BoW)")
    results_bow.append((name, accuracy))

# Addestramento e valutazione con TF-IDF
print("\n=== Risultati con TF-IDF ===\n")
for name, model in models.items():
    _, accuracy = train_and_evaluate(model, X_train_tfidf, X_test_tfidf, y_train, y_test, f"{name} (TF-IDF)")
    results_tfidf.append((name, accuracy))

In [ ]:
# Confronto dei risultati
results_df = pd.DataFrame({
    'Modello': [r[0] for r in results_bow],
    'Accuracy (BoW)': [r[1] for r in results_bow],
    'Accuracy (TF-IDF)': [r[1] for r in results_tfidf]
})

# Visualizzazione dei risultati
plt.figure(figsize=(12, 6))
results_df.set_index('Modello').plot(kind='bar')
plt.title('Confronto delle Performance dei Modelli')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tabella dei risultati
results_df

## 6. Ottimizzazione del Modello Migliore

Basandoci sui risultati precedenti, selezioniamo il modello con le migliori performance e ottimizziamo i suoi iperparametri utilizzando GridSearchCV.

In [ ]:
# Supponiamo che SVM con TF-IDF abbia ottenuto i migliori risultati
# Ottimizziamo i suoi iperparametri

# Definizione dei parametri da testare
param_grid = {
    'C': [0.1, 1, 10, 100],
    'loss': ['hinge', 'squared_hinge'],
    'dual': [True, False]
}

# Inizializzazione di GridSearchCV
grid_search = GridSearchCV(LinearSVC(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Addestramento
grid_search.fit(X_train_tfidf, y_train)

# Migliori parametri
print(f"Migliori parametri: {grid_search.best_params_}")
print(f"Miglior accuracy in cross-validation: {grid_search.best_score_:.4f}")

# Valutazione sul test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy sul test set: {accuracy:.4f}")
print("Classification Report:")
print(report)

# Matrice di confusione
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=sorted(y.unique()), 
            yticklabels=sorted(y.unique()))
plt.title('Matrice di Confusione - Modello Ottimizzato')
plt.ylabel('Valore Reale')
plt.xlabel('Valore Predetto')
plt.tight_layout()
plt.show()

## 7. Analisi delle Feature Importanti

Analizziamo quali parole o termini sono più importanti per ciascuna categoria. Questo può fornire insights preziosi sul linguaggio utilizzato dai clienti per diverse tipologie di richieste.

In [ ]:
# Funzione per visualizzare le parole più importanti per ogni categoria
def plot_important_features(vectorizer, model, class_names, n_top_features=10):
    feature_names = vectorizer.get_feature_names_out()
    
    # Per SVM, i coefficienti rappresentano l'importanza delle feature
    if hasattr(model, 'coef_'):
        coefs = model.coef_
    else:
        # Per altri modelli, potremmo dover usare approcci diversi
        print("Il modello non supporta l'estrazione diretta dell'importanza delle feature.")
        return
    
    plt.figure(figsize=(15, 20))
    
    for i, class_name in enumerate(class_names):
        # Otteniamo i coefficienti per questa classe
        class_coefs = coefs[i]
        
        # Troviamo le feature più importanti (con i coefficienti più alti)
        top_positive_indices = np.argsort(class_coefs)[-n_top_features:]
        top_positive_features = [feature_names[j] for j in top_positive_indices]
        top_positive_values = [class_coefs[j] for j in top_positive_indices]
        
        # Creiamo un subplot per questa classe
        plt.subplot(len(class_names), 1, i+1)
        plt.barh(range(n_top_features), top_positive_values[::-1])
        plt.yticks(range(n_top_features), top_positive_features[::-1])
        plt.title(f'Top {n_top_features} parole per la categoria "{class_name}"')
        plt.tight_layout()
    
    plt.tight_layout(pad=3.0)
    plt.show()

# Visualizzazione delle parole più importanti
plot_important_features(tfidf_vectorizer, best_model, sorted(y.unique()))

## 8. Applicazione del Modello a Nuovi Dati

Ora che abbiamo un modello addestrato e ottimizzato, possiamo utilizzarlo per classificare nuove richieste dei clienti. Creiamo una funzione che prende in input un testo e restituisce la categoria prevista.

In [ ]:
# Funzione per classificare nuove richieste
def classify_request(text, model, vectorizer):
    # Preprocessing del testo
    processed_text = preprocess_text(text)
    
    # Trasformazione in feature
    features = vectorizer.transform([processed_text])
    
    # Predizione
    category = model.predict(features)[0]
    
    # Probabilità (se disponibili)
    if hasattr(model, 'predict_proba'):
        probabilities = model.predict_proba(features)[0]
        confidence = probabilities.max()
    else:
        # Per modelli che non supportano predict_proba, come LinearSVC
        # Possiamo usare decision_function come approssimazione
        if hasattr(model, 'decision_function'):
            decision_scores = model.decision_function(features)[0]
            confidence = np.abs(decision_scores).max() / np.sum(np.abs(decision_scores))
        else:
            confidence = None
    
    return {
        'categoria': category,
        'confidenza': confidence
    }

# Testiamo la funzione con alcuni esempi
esempi = [
    "Non riesco a configurare la mia email sul nuovo telefono, potete aiutarmi?",
    "Ho ricevuto una fattura con importi non corretti, vorrei un chiarimento.",
    "Sono molto deluso dal vostro servizio, continuo ad avere problemi di connessione.",
    "Vorrei sapere se avete offerte speciali per i clienti esistenti.",
    "Il tecnico doveva venire stamattina ma non si è presentato, è inaccettabile!"
]

for esempio in esempi:
    risultato = classify_request(esempio, best_model, tfidf_vectorizer)
    print(f"Testo: {esempio}")
    print(f"Categoria prevista: {risultato['categoria']}")
    if risultato['confidenza'] is not None:
        print(f"Confidenza: {risultato['confidenza']:.4f}")
    print("---")

## 9. Implementazione di un Sistema di Classificazione in Produzione

In un ambiente di produzione, potremmo voler salvare il modello addestrato e il vectorizer per utilizzarli successivamente. Ecco come farlo:

In [ ]:
import joblib

# Salvataggio del modello e del vectorizer
joblib.dump(best_model, 'modello_classificazione.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

# Per caricare il modello e il vectorizer in un'applicazione di produzione:
# model = joblib.load('modello_classificazione.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')

## 10. Conclusioni e Considerazioni Finali

In questo notebook, abbiamo sviluppato un sistema di classificazione automatica delle richieste dei clienti utilizzando tecniche di NLP. Abbiamo:

1. Creato un dataset di esempio di richieste di customer care
2. Preprocessato i testi per l'analisi NLP
3. Implementato e confrontato diversi approcci di classificazione (Naive Bayes, Regressione Logistica, SVM, Random Forest)
4. Valutato le performance dei modelli utilizzando metriche come accuracy e classification report
5. Ottimizzato il modello migliore utilizzando GridSearchCV
6. Analizzato le feature più importanti per ciascuna categoria
7. Creato una funzione per classificare nuove richieste
8. Mostrato come salvare il modello per un'implementazione in produzione

### Considerazioni per l'Implementazione Reale

In un contesto aziendale reale, ci sono ulteriori considerazioni da tenere in mente:

- **Qualità e Quantità dei Dati**: Un dataset più ampio e rappresentativo porterebbe a performance migliori. Idealmente, si dovrebbero utilizzare migliaia di esempi reali per ciascuna categoria.

- **Evoluzione del Linguaggio**: Il linguaggio dei clienti evolve nel tempo, così come i prodotti e i servizi dell'azienda. È importante aggiornare periodicamente il modello con nuovi dati.

- **Multilingualità**: Se l'azienda serve clienti in diverse lingue, potrebbe essere necessario sviluppare modelli specifici per ciascuna lingua o utilizzare approcci multilingue.

- **Integrazione con i Sistemi Esistenti**: Il sistema di classificazione dovrebbe integrarsi con i sistemi CRM, ticketing o di gestione delle email esistenti.

- **Supervisione Umana**: È consigliabile mantenere un certo grado di supervisione umana, specialmente per richieste classificate con bassa confidenza o per categorie particolarmente critiche.

- **Feedback Loop**: Implementare un meccanismo per raccogliere feedback dagli operatori sulle classificazioni errate, utilizzando questi dati per migliorare continuamente il modello.

- **Approcci Più Avanzati**: Per casi d'uso più complessi, si potrebbero considerare approcci più avanzati come l'utilizzo di modelli pre-addestrati (BERT, GPT) o tecniche di transfer learning.

La classificazione automatica delle richieste dei clienti può portare a significativi miglioramenti nell'efficienza operativa e nella qualità del servizio, permettendo agli operatori di concentrarsi sugli aspetti più complessi e a valore aggiunto del customer care.